# Set Membership Coursework

In [18]:
import random
import string
import timeit
from bitarray import bitarray

## 1. Implementeation

### Sequential Search 

In [3]:
a = "sddfd"
type(a)

str

In [19]:
class SequentialSearchSet():
    
    def __init__(self):
        self.items = []           
        
    def insertElement(self, element):
        inserted = False
        if type(element) == str:
            self.items.append(element)
            inserted = True
        return inserted   

    def searchElement(self, element):
        found = False     
        for item1 in self.items:
            if item1 == element:
                found = True
                return found
        return found   

Time Complexity: 
- Insert an element is O(n^2)
- Search an element is O(n)

Space Complexity: 
- O(n)

### Binary Search Tree

In [5]:
class Node:
    
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def get(self, element):
        if self.value == element:
            return True
        elif element < self.value and self.left:
            return self.left.get(element)
        elif element > self.value and self.right:
            return self.right.get(element)
        return False

    def put(self, element):
        if self.value == None:
            self.value = element
            return True
        elif element == self.value:
            self.value = element
            return True
        elif element < self.value:
            if self.left is None:
                self.left = Node(element)
                return True
            else: 
                return self.left.put(element)
        elif element > self.value:
            if self.right is None:
                self.right = Node(element)
                return True
            else: 
                return self.right.put(element)
        return False

class BinarySearchTreeSet:
    
    def __init__(self):
        self.bst = Node(None)

    def insertElement(self, element):
        inserted = False
        inserted = self.bst.put(element) 
        return inserted

    def searchElement(self, element):     
        found = False 
        found = self.bst.get(element)
        return found

Time Complexity:

Insert: 
- Average: O(log n)
- Worst: O(n)

Search:
- Average: O(log n)
- Worst: O(n)

Space Complexity:
- O(n)

### Balanced Search Tree

In [6]:
class BalancedSearchTreeSet:
    R = True
    B = False
    # R refers to Red; B refers to Black
    
    class Nodes:
        def __init__(self, val, colour):
            self.val = val
            self.colour = colour
            self.left = None
            self.right = None
            
    def __init__(self):
        self.root = None
        
    def searchElement(self, element):
        found = False
        found = self.search(self.root, element)
        return found
    
    def search(self, node, val):
        if node is None:
            return False
        else:
            if node.val == val:
                return True
            elif node.val > val:
                return self.search(node.left, val)
            else:
                return self.search(node.right, val)
    
    def insertElement(self, element):
        inserted = False
        if self.root is None:
            self.root = self.Nodes(element, self.B)
            inserted = True
        else:
            self.root, inserted = self.insert(self.root, element)
            self.root.colour = self.B
        return inserted

    def insert(self, node, val):
        if node is None:
            return self.Nodes(val, self.R), True
        if val < node.val:
            node.left, inserted = self.insert(node.left, val)
        elif val > node.val:
            node.right, inserted = self.insert(node.right, val)
        else:
            # value already exists, do nothing
            return node, False

        # perform rotations and colour flips
        if self.isR(node.right) and not self.isR(node.left):
            node = self.leftRotate(node)
        if self.isR(node.left) and self.isR(node.left.left):
            node = self.rightRotate(node)
        if self.isR(node.left) and self.isR(node.right):
            self.flipColour(node)

        return node, inserted

    def isR(self, node):
        if node is None:
            return False
        return node.colour == self.R
    
    def leftRotate(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def rightRotate(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def flipColour(self, node):
        node.colour = not node.colour
        node.left.colour = self.B
        node.right.colour = self.B

Time Complexity: 

- Insert an element is O(log n)
- Search an element is O(log n)

Space Complexity: 

- O(n)

### 2-3 Search Tree

In [7]:
class Node2:
    def __init__(self, values, parent = None):
        self.values = [values]
        self.parent = parent
        self.child = []
    
    def is_leaf(self):
        return len(self.child) == 0
    
    def __lt__(self, other):
        return self.values[0] < other.values[0]
    
    def add(self, new_node):
        for child in new_node.child:
            child.parent = self
        self.values.extend(new_node.values)
        self.values.sort()
        self.child.extend(new_node.child)
        if len(self.child)>1:
            self.child.sort()
        if len(self.values)>2:
            self.split()
    
    def _insert(self, new_node):
        if self.is_leaf() == True:
            if new_node.values[0] not in self.values:
                self.add(new_node)
                return True
            return False
        elif new_node.values[0] > self.values[-1]:
            return self.child[-1]._insert(new_node)
        else:
            for i in range(0, len(self.values)):
                if new_node.values[0] < self.values[i]:
                    return self.child[i]._insert(new_node)
                    
    
    def split(self):
        left_child = Node2(self.values[0], self)
        right_child = Node2(self.values[2], self)
        if len(self.child) != 0:
            self.child[0].parent = left_child
            self.child[1].parent = left_child
            self.child[2].parent = right_child
            self.child[3].parent = right_child
            left_child.child = [self.child[0], self.child[1]]
            right_child.child = [self.child[2], self.child[3]]
        
        #promote middle node
        self.child = [left_child]
        self.child.append(right_child)
        self.values = [self.values[1]]

        if self.parent is not None:
            if self in self.parent.child:
                self.parent.child.remove(self)
            self.parent.add(self)
        else:
            left_child.parent = self
            right_child.parent = self

    def _find(self, element):
        if element in self.values:
            return True
        elif self.is_leaf() == True:
            return False
        elif element > self.values[-1]:
            return self.child[-1]._find(element)
        else:
            for i in range (0, len(self.values)):
                if element < self.values[i]:
                    return self.child[i]._find(element)
        
class TwoThreeBSTSet:
    def __init__(self):
        self.root = None
    
    def insertElement(self, element):
        inserted = False
        new_node = Node2(element)
        if self.root is None:
            self.root = new_node
        else:
            inserted = self.root._insert(new_node)
            if not inserted:
                return False
            #re-establishes the root to the top of the tree
            while self.root.parent is not None:
                self.root = self.root.parent
        return True 
    
    def searchElement(self, element):
        found = False
        found =self.root._find(element)
        return found

Time Complexity:
- Insert an element is O(log n)
- Search an element is O(log n)

Space Complexity:
- O(n)

### Bloom Filter

In [27]:
class BloomFilterSet:
    
    def __init__(self):
        self.HashFunctions = None
        self.hashNum = 0
        self.size = 0
        self.bitArray = None
    
    def initVariables(self, ItemNum, FPProb, HashFunctions):
        # ItemNum : int, Number of items expected to be stored in bloom filter
        # FPProb : float, False Positive probability in decimal
        # HashFunctions: list, number of hash functions to use
        self.HashFunctions = HashFunctions
        self.hashNum = len(HashFunctions)
        # Total size of bit array
        self.size = self.getSize(ItemNum, FPProb)
        # Bit array of given size, and initialize all bits as 0 (False)
        self.bitArray = bitarray(self.size)
        self.bitArray.setall(0)

    def insertElement(self, element):
        inserted = False
        tempIndex = []
        for function in self.HashFunctions:
            index = function(element) % self.size
            self.bitArray[index] = True
            tempIndex.append(self.bitArray[index])
        if False not in tempIndex:
            inserted = True
        return inserted

    def searchElement(self, element):
        found = False
        count = 0
        for function in self.HashFunctions:
            index = function(element) % self.size
            if not self.bitArray[index] == False:
                count += 1
            else:
                return found
        if count == len(self.HashFunctions):
            found = True
        return found

    def getSize(self, n, p):
        # m = -(n * lg(p)) / (lg(2)^2)
        # n : number of items expected to be stored in filter
        # p : False Positive probability in decimal
        m = -(n * self.myLog(p)) / (self.myLog(2) ** 2)
        return int(m)
    
    def myLog(self, n):
        if n <= 0:
            raise ValueError("logarithm is undefined for non-positive values")
        if n == 1:
            return 0
        if n < 1:
            # log(xn) = -log(1/n) for n < 1
            return -self.myLog(1 / n)
        # Use the taylor series approximation for log(n) where |n-1| < 1
        taylor = (n - 1) / float(n + 1)
        term = taylor
        total = taylor
        num = 1
        while abs(term) > 1e-15:
            num += 2
            term *= taylor * taylor / float(num * (num - 1))
            total += term
        return 2 * total

In [28]:
def builtinHash(string):
    return hash(string)

def bernsteinHash(string):
    hashValue = 5381
    for char in string:
        hashValue = ((hashValue << 5) + hashValue) + ord(char)
    return hashValue

def fnvHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue = (hashValue * 16777619) ^ ord(char)
    return hashValue

def jenkinsHash(string):
    hashValue = 0
    for char in string:
        hashValue += ord(char)
        hashValue += (hashValue << 10)
        hashValue ^= (hashValue >> 6)
    hashValue += (hashValue << 3)
    hashValue ^= (hashValue >> 11)
    hashValue += (hashValue << 15)
    return hashValue

def djb2Hash(string):
    hashValue = 5381
    for char in string:
        hashValue = ((hashValue << 5) + hashValue) + ord(char)
    return hashValue

def sdbmHash(string):
    hashValue = 0
    for char in string:
        hashValue = ord(char) + (hashValue << 6) + (hashValue << 16) - hashValue
    return hashValue

def fnv1aHash(string):
    hashValue = 2166136261
    for char in string:
        hashValue ^= ord(char)
        hashValue *= 16777619
    return hashValue

In [38]:
Alphabets = list(string.ascii_letters)
randomword = []
for a in range(10000):
    temp = ''.join(random.choice(Alphabets) for t in range(random.randint(100, 200)))
    randomword.append(temp)

functions = [builtinHash, bernsteinHash, fnvHash, jenkinsHash, 
             djb2Hash, sdbmHash, fnv1aHash]
bf = BloomFilterSet()
bf.initVariables(10000, 0.01, functions)
testnum = 0
for ws in randomword:
    if bf.searchElement(ws) == True:
        testnum += 1

print("The False-Positive-Rate: ", testnum / len(randomword))

The False-Positive-Rate:  0.0


Time Complexity: 
- Creating the Bloom filter is O(kn)
- Insert and Search an element: O(k)
- k is the number of hash functions and n is the number of elements

Space Complexity: 
- O(kb), where k is the number of hash functions and b is the bits number

## 2. Experimentally Evaluation

### Using Real Data

In [20]:
def ReadFile(filepath):
    with open(filepath) as f:
        for line in f:
            words = line.strip().split()
    return words

In [21]:
words1 = ReadFile("./data/testfiles/test1-mobydick.txt")
words2 = ReadFile("./data/testfiles/test2-warpeace.txt")
words3 = ReadFile("./data/testfiles/test3-dickens.txt")

files = {"'test1-mobydick.txt'": words1, "'test2-warpeace.txt'": words2,
         "'test3-dickens.txt'": words3}
print(len(words1), len(words2), len(words3))

209329 564236 5149661


In [22]:
wordsforsearch = []
with open("./data/testfiles/test-search.txt") as fsearch:
    for line in fsearch:
        wordsforsearch.append(line.strip())
        
len(wordsforsearch)

544

In [24]:
hashFunctions = [builtinHash, bernsteinHash, fnvHash, jenkinsHash, 
                 djb2Hash, sdbmHash, fnv1aHash]

#### The total amount of time of all insert operations

In [25]:
def InsertTime(classname, datadict):
    res = []
    classlist = []
    
    for key in datadict.keys():
        tempdata = datadict[key]
        algorithm = classname()
        if str(classname) == str(BloomFilterSet):
            algorithm.initVariables(len(tempdata), 0.01, hashFunctions)

        seen = {}
        InsertStart = timeit.default_timer()
        for word in tempdata:
            if word is not None and word not in seen:
                seen[word] = True
                inserted = algorithm.insertElement(word)
                res.append(inserted)
        InsertEnd = timeit.default_timer()
        InsertTime = InsertEnd - InsertStart
        print("Insert time taken for", key, "is :", InsertTime)
        classlist.append(algorithm)
    return classlist, res

In [29]:
# Sequential Search
SEQs, res = InsertTime(SequentialSearchSet, files)
res

Insert time taken for 'test1-mobydick.txt' is : 0.03290649999996731
Insert time taken for 'test2-warpeace.txt' is : 0.07714809999993122
Insert time taken for 'test3-dickens.txt' is : 0.5778016000000434


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [220]:
# Binary Search Tree
BSTs, res = InsertTime(BinarySearchTreeSet, files)
res

Insert time taken for 'test1-mobydick.txt' is : 0.17177070000616368
Insert time taken for 'test2-warpeace.txt' is : 0.7194921000045724
Insert time taken for 'test3-dickens.txt' is : 1.3332328999968013


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [216]:
# Balanced Search Tree
RBTs, res = InsertTime(BalancedSearchTreeSet, files)
res

Insert time taken for 'test1-mobydick.txt' is : 0.42829019999771845
Insert time taken for 'test2-warpeace.txt' is : 0.3952906000049552
Insert time taken for 'test3-dickens.txt' is : 3.344172199998866


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [222]:
# Balanced Search Tree -- 2-3 Search Tree
TTSTs, res = InsertTime(TwoThreeBSTSet, files)
res

Insert time taken for 'test1-mobydick.txt' is : 0.8376441000000341
Insert time taken for 'test2-warpeace.txt' is : 0.32367539999540895
Insert time taken for 'test3-dickens.txt' is : 4.147186800000782


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [217]:
# Bloom Filter
BFs, res = InsertTime(BloomFilterSet, files)
res

Insert time taken for 'test1-mobydick.txt' is : 0.297156200002064
Insert time taken for 'test2-warpeace.txt' is : 0.32849069999792846
Insert time taken for 'test3-dickens.txt' is : 1.8756322999979602


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

#### The total amount of time of search operations

In [1]:
def SearchTime(objects, searchdata):
    res = []
    for obj in objects:
        tempres = []
        SearchStart = timeit.default_timer()
        for ws in searchdata:
            if ws is not None:
                searched = obj.searchElement(ws)
            else:
                searched = None
            tempres.append(searched)
        SearchEnd = timeit.default_timer()
        SearchTime = SearchEnd - SearchStart
        print("Search time taken for 'test-search.txt' in file", 
              objects.index(obj)+1, "is :", SearchTime)
        res.append(tempres)
    return res

In [224]:
# Sequential Search Tree
SearchTime(SEQs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.133109999995213
Search time taken for 'test-search.txt' in file 2 is : 0.09881110000424087
Search time taken for 'test-search.txt' in file 3 is : 0.3563752000045497


[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,


In [225]:
# Binary Search Tree
SearchTime(BSTs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.0028355999966152012
Search time taken for 'test-search.txt' in file 2 is : 0.002892299999075476
Search time taken for 'test-search.txt' in file 3 is : 0.002479700000549201


[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,


In [226]:
# Balanced Search Tree
SearchTime(RBTs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.0051075999945169315
Search time taken for 'test-search.txt' in file 2 is : 0.0020838999989791773
Search time taken for 'test-search.txt' in file 3 is : 0.002711000000999775


[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,


In [229]:
# Balanced Search Tree -- 2-3 Search Tree
SearchTime(TTSTs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.0046432000017375685
Search time taken for 'test-search.txt' in file 2 is : 0.004830300000321586
Search time taken for 'test-search.txt' in file 3 is : 0.005131000005349051


[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,


In [228]:
# Bloom Filter
SearchTime(BFs, wordsforsearch)

Search time taken for 'test-search.txt' in file 1 is : 0.004998399999749381
Search time taken for 'test-search.txt' in file 2 is : 0.005291099994792603
Search time taken for 'test-search.txt' in file 3 is : 0.009204599999065977


[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,


### Using Synthetic Data

### different conditions:
- Duplicate cases
- Longer String length
- Adding numbers and symbols in each string
- String list in ascending/descending order
- Missing values
- ......

In [15]:
class TestDataGenerator():

    def __init__(self):
        self.syntheticData = []
        self.strLen = 0
        self.percentage = 0.0

    def setParams(self, strLen, percentage):
        self.strLen = strLen
        self.percentage = percentage
    
    def generateData(self, size):
        data, data1, data2 = [], [], []
        seen1, seen2, seen3 = {}, {}, {}
        alphabets = list(string.ascii_letters)
        charlist = string.printable

        # Normal Case
        while len(data) < size:
            temp = ''.join(random.choice(alphabets) for _ in range(random.randint(4, 10)))
            if temp not in seen1:
                seen1[temp] = True
                data.append(temp)

        # Long String
        while len(data1) < size:
            temp1 = ''.join(random.choice(alphabets) for _ in range(self.strLen))
            if temp1 not in seen2:
                seen2[temp1] = True
                data1.append(temp1)

        # Special String
        while len(data2) < size:
            temp2 = ''.join(random.sample(charlist, random.randint(4, 10)))
            if temp2 not in seen3:
                seen3[temp2] = True
                data2.append(temp2)

        # Ascending Order
        data3 = self.mergeSort(data.copy())

        # String with missing values and duplicate values
        num6 = round(size * 0.2)
        temp6 = data[ : num6]
        data5 = [None if i >= num6 and random.random() < self.percentage else data[i] for i in range(size)]
        data6 = [random.choice(temp6) if i >= num6 and random.random() < self.percentage else data[i] for i in range(size)]

        # Descending Order
        data4 = data3[::-1]

        self.syntheticData.append(data)
        self.syntheticData.append(data1)
        self.syntheticData.append(data2)
        self.syntheticData.append(data3)
        self.syntheticData.append(data4)
        self.syntheticData.append(data5)
        self.syntheticData.append(data6)
        return self.syntheticData

    def mergeSort(self, arr):
        n = len(arr)
        size = 1
        while size < n:
            for i in range(0, n, 2*size):
                left = arr[i : i+size]
                right = arr[i+size : i+2*size]
                merged = []
                l = r = 0
                while l < len(left) and r < len(right):
                    if left[l] < right[r]:
                        merged.append(left[l])
                        l += 1
                    else:
                        merged.append(right[r])
                        r += 1
                merged += left[l:]
                merged += right[r:]
                arr[i : i+len(merged)] = merged
            size *= 2
        return arr

In [16]:
dataGenerate = TestDataGenerator()
dataGenerate.setParams(100, 0.6)
SyntheticDataList = dataGenerate.generateData(100000)
SyntheticDataList

[['BwwWOuaWjD',
  'MPGIfsgD',
  'jzLDtwHa',
  'iWPNMmSY',
  'fPrgESnVL',
  'KblksWdj',
  'IPPNj',
  'GOexbH',
  'caotAkJf',
  'MrqxGsCtjs',
  'QRyMoeECQj',
  'kUcNbTXrR',
  'EehyCO',
  'HtAuo',
  'vuxDJzAeDx',
  'FgBRGS',
  'sLvamev',
  'dpDS',
  'kRuQv',
  'OnsYvhywP',
  'fMaO',
  'pTTj',
  'gHMG',
  'fbEKXdIWVG',
  'AoWP',
  'WSrqkMWTk',
  'tGTtPWy',
  'SLNXcqZdIs',
  'XoIDtlyc',
  'oKxW',
  'eFJCmDxv',
  'uVhPindF',
  'JVwB',
  'lTbUFAAb',
  'DyjLD',
  'BycZOIkOY',
  'CirfuT',
  'YIUQA',
  'YCsSd',
  'wMFFLV',
  'ZRIyYk',
  'zGcvMVs',
  'wFzif',
  'SSeXEHjJ',
  'zoluJmv',
  'JgIxjMrcQ',
  'yqGXY',
  'OKubay',
  'gKrowKms',
  'PphwWBpiUw',
  'RdTVhLwJng',
  'BneTs',
  'KCiMPVlr',
  'AorPniK',
  'hMJsoU',
  'KJtu',
  'evEgza',
  'LStdW',
  'fQelM',
  'mRMEdb',
  'zUAV',
  'nujZUDkfCk',
  'fCgfAyFiC',
  'NQrJtX',
  'VDsBoUBWuM',
  'vMHDyiHiFb',
  'wFdiFy',
  'etKxbFWD',
  'eWSHY',
  'qLMq',
  'segKER',
  'AFRYO',
  'aXduxNYw',
  'INtyl',
  'rYwE',
  'olakInrsJ',
  'iCemDE',
  'ZOdV',
 

In [21]:
SyntheticDataNames = ["Normal", "Long String", "Special String", "Ascending Order", "Descending Order", 
                      "String with missing values", "String with duplicate values"]

In [18]:
synSearchList = []
for datalist in SyntheticDataList[0 : 3]:
    num = round(len(datalist) * 0.3)
    templist = []
    seen = {}
    # Choose from list
    while len(templist) < num:
        tempdata = random.choice(datalist)
        if tempdata not in seen:
            seen[tempdata] = True
            templist.append(tempdata)
    # Generate new values
    falsenum = round(len(datalist) * 0.2)
    for _ in range(falsenum):
        fdata = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(8, 12))) 
        templist.append(fdata)
    synSearchList.append(templist)

for _ in range(4):
    synSearchList.append(synSearchList[0].copy())
synSearchList

[['KNFUE',
  'CEuqBQ',
  'ilbMJKHIuv',
  'nxCsF',
  'CkxRqEUKL',
  'uJVAlK',
  'npDwzhNP',
  'LtPXON',
  'qVJstREZW',
  'yYCCMw',
  'bohZECccbj',
  'yYCiwDCoC',
  'MLoNmWI',
  'nrGfHbBBW',
  'KhLEI',
  'xeIftKtt',
  'NkxGxmq',
  'GsiNNHq',
  'vxFZ',
  'FKvtOAcMhX',
  'HprXqJthU',
  'HnczYEetU',
  'DBGQP',
  'HZvsoKiFnw',
  'jKRmEDLeEb',
  'QMzRpe',
  'cBfALRy',
  'JgpemKavC',
  'RFLGCyRyaR',
  'WoJMV',
  'hiPu',
  'ubjKSs',
  'QhedpRoiB',
  'LyeWhTrVJi',
  'QptAbKtYmR',
  'gezOv',
  'bKRYjwnaP',
  'luBV',
  'Hzuz',
  'LRMXk',
  'HgdLkwA',
  'PkHsFdIBud',
  'nzbnFGCn',
  'vnmmhgk',
  'sEJm',
  'ngQQKqQEK',
  'JYJi',
  'qSUn',
  'ugRffg',
  'peLAK',
  'ioQLi',
  'RryPhWtDwj',
  'nPIxm',
  'vyvOVmC',
  'ArIdPNkH',
  'YUjLgkrMCn',
  'PFEy',
  'tDbRxsgYzS',
  'HfBoVoY',
  'fWUz',
  'gAac',
  'UVuSTEIAus',
  'SAvOLwszs',
  'eaitesYU',
  'dXiGtYN',
  'bGYNV',
  'znIFU',
  'PpCXYH',
  'quBJsiAKh',
  'ARhtRLUcZ',
  'VWBhP',
  'XOIMjK',
  'QXkYUYmKCB',
  'NNXbMlTB',
  'yWRCHHKzXm',
  'dKyrkA',
 

#### Insert Time Taken

In [19]:
def InsertTime2(classname, datalist):
    # Remove Duplicates
    seen = {}
    uniqueDatas = []
    for e in datalist:
        if e not in seen:
            seen[e] = True
            uniqueDatas.append(e)
    
    algorithm = classname()
    if str(classname) == str(BloomFilterSet):
        algorithm.initVariables(len(datalist), 0.01, hashFunctions)

    InsertStart = timeit.default_timer()
    res = []
    for word in uniqueDatas:
        # Remove Missing Values
        if word is not None:
            inserted = algorithm.insertElement(word)
            res.append(inserted)
    InsertEnd = timeit.default_timer()
    InsertTime = InsertEnd - InsertStart
    print("Insert time taken is :", InsertTime)
    return algorithm, res

In [4]:
SEQSyn, BSTSyn, RBTSyn, TTSTSyn, BTSyn = [], [], [], [], []
InsertRes1, InsertRes2, InsertRes3, InsertRes4, InsertRes5 = [], [], [], [], []
for i in range(len(SyntheticDataList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    SEQ, res1 = InsertTime2(SequentialSearchSet, SyntheticDataList[i])
    SEQSyn.append(SEQ)
    InsertRes1.append(res1)
    print("Binary Search Tree: ")
    BST, res2 = InsertTime2(BinarySearchTreeSet, SyntheticDataList[i])
    BSTSyn.append(BST)
    InsertRes2.append(res2)
    print("Balanced Search Tree: ")
    RBT, res3 = InsertTime2(BalancedSearchTreeSet, SyntheticDataList[i])
    RBTSyn.append(RBT)
    InsertRes3.append(res3)
    print("2-3 Search Tree: ")
    TTST, res4 = InsertTime2(TwoThreeBSTSet, SyntheticDataList[i])
    TTSTSyn.append(TTST)
    InsertRes4.append(res4)
    print("Bloom Filter: ")
    BT, res5 = InsertTime2(BloomFilterSet, SyntheticDataList[i])
    BTSyn.append(BT)
    InsertRes5.append(res5)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [187]:
for l in range(len(SyntheticDataList)):
    print(InsertRes1[l][:20], InsertRes2[l][:20], InsertRes3[l][:20], InsertRes4[l][:20], InsertRes5[l][:20])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


### Search Time Taken

In [185]:
def SearchTime2(obj, searchdata):
    res = []
    SearchStart = timeit.default_timer()
    for ws in searchdata:
        if ws is not None:
            searched = obj.searchElement(ws)
        else:
            searched = None
        res.append(searched)
    SearchEnd = timeit.default_timer()
    SearchTime = SearchEnd - SearchStart
    print("Search time taken is :", SearchTime)
    return res

In [188]:
SearchRes1, SearchRes2, SearchRes3, SearchRes4, SearchRes5 = [], [], [], [], []
for i in range (len(synSearchList)):
    print("For", SyntheticDataNames[i], "Case: ")
    print("Sequential Search: ")
    sres1 = SearchTime2(SEQSyn[i], synSearchList[i])
    SearchRes1.append(sres1)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Binary Search Tree: ")
    sres2 = SearchTime2(BSTSyn[i], synSearchList[i])
    SearchRes2.append(sres2)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Balanced Search Tree: ")
    sres3 = SearchTime2(RBTSyn[i], synSearchList[i])
    SearchRes3.append(sres3)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    sres4 = SearchTime2(RBTSyn[i], synSearchList[i])
    SearchRes4.append(sres4)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Bloom Filter: ")
    sres5 = SearchTime2(BTSyn[i], synSearchList[i])
    SearchRes5.append(sres5)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("                                                 ")

For Normal Case: 
Sequential Search: 
Search time taken is : 108.61616439999489
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bloom Filter: 
Search time taken is : 0.5147987000018475
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                                 


In [189]:
for ls in range(len(synSearchList)):
    print(SearchRes1[ls][:20], SearchRes2[ls][:20], SearchRes3[ls][:20], SearchRes4[ls][:20], SearchRes5[ls][:20])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


# --------

In [60]:
# missing value checker

# def isNone(list):
#     i = 0
#     lst_iter = iter(list)
#     while True:
#         element = next(lst_iter,"end")
#         if element == "end":
#             break
#         if element is None:
#             i += 1
#     print("The file contains %d None values" % i)

# testList = [1,2,3,None,4,5,6,7,8,9,None]

# isNone(words1)
# isNone(words2)
# isNone(words3)
# isNone(testList)

The file contains 0 None values
The file contains 0 None values
The file contains 0 None values
The file contains 2 None values


In [ ]:
# rbt = BalancedSearchTreeSet()
# rbt.insertElement('b')
# rbt.insertElement('a')
# rbt.insertElement('c')

# print(rbt.searchElement('b'))
# print(rbt.searchElement('a'))
# print(rbt.searchElement('c'))

True
True
True


Sequential Search

In [5]:
# for key in files.keys():
#     seqInsertStart = timeit.default_timer()
#     for word in files[key]:
#         SEQs.insertElement(word)
#     seqInsertEnd = timeit.default_timer()
#     seqInsertTime = seqInsertEnd - seqInsertStart
#     print("Insert time taken for ", key, "is : ", seqInsertTime)

Sequential Search

In [ ]:
# SeqSearchStart = timeit.default_timer()
# for ws in wordsforsearch:
#     SEQs.searchElement(ws)
# SeqSearchEnd = timeit.default_timer()
# SeqSearchTime = SeqSearchEnd - SeqSearchStart
# print("Search time taken for 'test-search.txt' is : ", SeqSearchTime)

Binary Search Tree

In [6]:
# for key in files.keys():
#     binaryInsertStart = timeit.default_timer()
#     for word in files[key]:
#         BSTs.insertElement(word)
#     binaryInsertEnd = timeit.default_timer()
    
#     binaryInsertTime = binaryInsertEnd - binaryInsertStart
#     print("Insert time taken for ", key, "is : ", binaryInsertTime)

Binary Search Tree

In [7]:
# BinarySearchStart = timeit.default_timer()
# for ws in wordsforsearch:
#     BSTs.searchElement(ws)
# BinarySearchEnd = timeit.default_timer()
# BinarySearchTime = BinarySearchEnd - BinarySearchStart
# print("Search time taken for 'test-search.txt' is : ", BinarySearchTime)

Balanced Search Tree

In [8]:
# for key in files.keys():
#     RedBlackInsertStart = timeit.default_timer()
#     for word in files[key]:
#         RBT.insertElement(word)
#     RedBlackInsertEnd = timeit.default_timer()
#     RedBlackInsertTime = RedBlackInsertEnd - RedBlackInsertStart
#     print("Insert time taken for ", key, "is : ", RedBlackInsertTime)

Balanced Search Tree

In [9]:
# RedBlackSearchStart = timeit.default_timer()
# for ws in wordsforsearch:
#     RBT.searchElement(ws)
# RedBlackSearchEnd = timeit.default_timer()
# RedBlackSearchTime = RedBlackSearchEnd - RedBlackSearchStart
# print("Search time taken for 'test-search.txt' is : ", RedBlackSearchTime)

Bloom Filter

In [10]:
# for key in files.keys():
#     BFInsertStart = timeit.default_timer()
#     for word in files[key]:
#         BF.insertElement(word)
#     BFInsertEnd = timeit.default_timer()
#     BFInsertTime = BFInsertEnd - BFInsertStart
#     print("Insert time taken for ", key, "is : ", BFInsertTime)

Bloom Filter

In [11]:
# BFSearchStart = timeit.default_timer()
# for word in wordsforsearch:
#     BF.searchElement(word)
# BFSearchEnd = timeit.default_timer()
# BFSearchTime = BFSearchEnd - BFSearchStart
# print("Search time taken for 'test-search.txt' is : ", BFSearchTime)

In [12]:
# a = './data/testfiles/test1-mobydick.txt'
# b = './data/testfiles/test2-warpeace.txt'
# c = './data/testfiles/test3-dickens.txt'
# insertFiles = [a,b,c]

# for file in insertFiles:
#     seqInsertStart = timeit.default_timer()
#     seqS.insertElement(file)
#     seqInsertEnd = timeit.default_timer()
#     seqInsertTime = seqInsertEnd - seqInsertStart
#     print(seqInsertTime)

In [13]:
# a = './data/testfiles/test1-mobydick.txt'
# b = './data/testfiles/test2-warpeace.txt'
# c = './data/testfiles/test3-dickens.txt'
# d = "./data/testfiles/test-search.txt"
# insertFiles = [a,b,c]
# for file in insertFiles:
#     seqS.insertElement(file)
#     SeqSearchStart = timeit.default_timer()
#     seqS.searchElement(d)
#     SeqSearchEnd = timeit.default_timer()
#     SeqSearchTime = SeqSearchEnd - SeqSearchStart
#     print(SeqSearchTime)

In [14]:
# a = './data/testfiles/test1-mobydick.txt'
# b = './data/testfiles/test2-warpeace.txt'
# c = './data/testfiles/test3-dickens.txt'
# insertFiles = [a,b,c]

# for file in insertFiles:
#     BSTInsertStart = timeit.default_timer()
#     rbt.insertElement(file)
#     BSTInsertEnd = timeit.default_timer()
#     BSTInsertTime = BSTInsertEnd - BSTInsertStart
#     print(BSTInsertTime)

In [15]:
# def insert_file():
#     a = './data/testfiles/test1-mobydick.txt'
#     b = './data/testfiles/test2-warpeace.txt'
#     c = './data/testfiles/test3-dickens.txt'
#     insertFiles = [a,b,c]
#     for file in insertFiles:
#         BSTInsertStart = timeit.default_timer()
#         bst.insertElement(file)
#         BSTInsertEnd = timeit.default_timer()
#         BSTInsertTime = BSTInsertEnd - BSTInsertStart
#         print(BSTInsertTime)

# def insert_real_data_test(words, bst):
#     BSTInsertStart = timeit.default_timer() 
#     for word in words:
#         bst.insertElement(word)
#     BSTInsertEnd = timeit.default_timer()
#     BSTInsertTime = BSTInsertEnd - BSTInsertStart
#     print("Insert time: ", BSTInsertTime)

# def search_words_in_set(words, bst):
#     BSTSearchStart = timeit.default_timer() 
#     for word in words:
#         bst.searchElement(word)
#     BSTSearchEnd = timeit.default_timer()
#     BSTSearchTime = BSTSearchEnd - BSTSearchStart
#     print("Search time (word already in set): ", BSTSearchTime)


# def search_words(words, bst):
#     BSTSearchStart = timeit.default_timer() 
#     for word in words:
#         bst.searchElement(word)
#     BSTSearchEnd = timeit.default_timer()
#     BSTSearchTime = BSTSearchEnd - BSTSearchStart
#     print("Search time (test-search words): ", BSTSearchTime)


# bst1 = BinarySearchTreeSet()
# bst2 = BinarySearchTreeSet()
# bst3 = BinarySearchTreeSet()

# insert_real_data_test(words, bst1)
# insert_real_data_test(words2, bst2)
# insert_real_data_test(words3, bst3)

# search_words_in_set(words, bst1)
# search_words_in_set(words2, bst2)
# search_words_in_set(words3, bst3)

# search_words(words, bst1)
# search_words(words2, bst2)
# search_words(words3, bst3)

In [16]:
# a = './data/testfiles/test1-mobydick.txt'
# b = './data/testfiles/test2-warpeace.txt'
# c = './data/testfiles/test3-dickens.txt'
# d = "./data/testfiles/test-search.txt"
# insertFiles = [a,b,c]

# for file in insertFiles:
#     rbt.insertElement(file)
#     BSTSearchStart = timeit.default_timer()
#     rbt.searchElement(d)
#     BSTSearchEnd = timeit.default_timer()
#     BSTSearchTime = BSTSearchEnd - BSTSearchStart
#     print(BSTSearchTime)

In [17]:
# # 1st version

# class TestDataGenerator():

#     def __init__(self):
#             self.syntheticData = []
            
#             pass           
            
#     # sythetic data -- integers:
#     def generateNum(self, size):
#         for _ in range(size):
#             item = random.randrange(0,1000000)
#             self.syntheticData.append(item)
#         print(self.syntheticData) # delete when it's finished
#         return self.syntheticData   
    
#     # sythetic data -- 4-letter string (randomly without real meanings):
#     def generateString(self,size):
#         letters = string.ascii_lowercase
#         for letter in letters:
#             for _ in range(size):
#                 letter1 = ''.join(random.choice(letters))
#                 letter2 = ''.join(random.choice(letters))
#                 letter3 = ''.join(random.choice(letters))
#                 stringGenerated = letter + letter1 + letter2 + letter3
#                 self.syntheticData.append(stringGenerated)
#         print(self.syntheticData) # delete when it's finished
#         return self.syntheticData

# toGenerate = TestDataGenerator()
# toSearch = TestDataGenerator()

# toGenerate.generateString(1)
# toSearch.generateNum(10)